# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("output_data/WeatherPy.csv")
weather_data.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,smithers,54.78,-127.17,30.20,80,40,9.17,CA,1609724376
1,rikitea,-23.12,-134.97,77.92,80,2,17.38,PF,1609724377
2,trairi,-3.28,-39.27,79.05,82,30,8.84,BR,1609724773
3,nyurba,63.28,118.33,-36.47,82,77,4.07,RU,1609724773
4,tuatapere,-46.13,167.68,63.00,67,90,8.99,NZ,1609724377


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [21]:
gmaps.configure(api_key=g_key)
figure_layout = {'width': '900px', 'height' : '600px'}

In [24]:
locations = weather_data[["Lat", "Lng"]].astype(float)
humidity = weather_data["Humidity"].astype(float)

fig = gmaps.figure(center=(40,0), zoom_level = 1.8, layout = figure_layout)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='600px', width='900px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [40]:
hotel_df = pd.DataFrame(weather_data.loc[((weather_data['Max Temp'] > 70) & (weather_data['Max Temp'] < 80) &
                                          (weather_data['Wind Speed'] < 10) & (weather_data['Cloudiness'] == 0) &
                                         (weather_data['Humidity'] > 65) & (weather_data['Humidity'] < 75)), :])
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,busselton,-33.65,115.33,71.01,51,0,7.00,AU,1609724379
16,arlit,18.74,7.39,73.22,21,0,5.55,NE,1609724776
51,saint-pierre,-21.34,55.48,75.20,73,0,8.05,RE,1609724387
77,rio grande,-32.03,-52.10,73.99,79,0,4.00,BR,1609724393
86,la orilla,17.98,-102.23,75.60,72,0,1.54,MX,1609724791
110,lafiagi,8.87,5.42,77.86,55,0,2.33,NG,1609724728
115,takoradi,4.88,-1.76,78.37,87,0,7.56,GH,1609724798
190,mar del plata,-38.00,-57.56,73.40,64,0,6.93,AR,1609724168
194,bonthe,7.53,-12.51,78.84,80,0,5.64,SL,1609724816
209,kayes,14.00,-11.00,74.71,21,0,5.99,ML,1609724820


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [34]:
hotel_df['Hotel Name'] = ''
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
51,saint-pierre,-21.34,55.48,75.20,73,0,8.05,RE,1609724387,
75,saint-joseph,-21.37,55.62,75.20,73,0,8.05,RE,1609724393,
86,la orilla,17.98,-102.23,75.60,72,0,1.54,MX,1609724791,
238,saint-philippe,-21.36,55.77,75.20,73,0,8.05,RE,1609724220,
405,umuarama,-23.77,-53.33,72.19,72,0,8.19,BR,1609724865,
515,umm lajj,25.02,37.27,70.61,66,0,7.87,SA,1609724491,
516,manzanillo,20.34,-77.12,71.82,74,0,3.11,CU,1609724491,
564,la mira,18.04,-102.33,74.50,71,0,2.42,MX,1609724902,


In [36]:
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]

    
    params["location"] = f"{lat},{lng}"

    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    hotel_name = requests.get(base_url, params=params)
    
    hotel_name = hotel_name.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except:
        pass

hotel_df
# need to adjust original weather dataframe to not use coordinates too close together. See the 3 very similar below.

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
51,saint-pierre,-21.34,55.48,75.20,73,0,8.05,RE,1609724387,"Villa Delisle, Hotel & Spa"
75,saint-joseph,-21.37,55.62,75.20,73,0,8.05,RE,1609724393,Village Sauvage
86,la orilla,17.98,-102.23,75.60,72,0,1.54,MX,1609724791,Hotel de Casablanca
238,saint-philippe,-21.36,55.77,75.20,73,0,8.05,RE,1609724220,Pinpin d'Amour
405,umuarama,-23.77,-53.33,72.19,72,0,8.19,BR,1609724865,HOTEL CAIUÁ
515,umm lajj,25.02,37.27,70.61,66,0,7.87,SA,1609724491,HP Red Sea Hotel
516,manzanillo,20.34,-77.12,71.82,74,0,3.11,CU,1609724491,Hostal Reliance (Casa Julio & Mayra)
564,la mira,18.04,-102.33,74.50,71,0,2.42,MX,1609724902,JP hotels & suites


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
